In [1]:
import pandas as pd
import time
import re
import datetime

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from konlpy.tag import Hannanum
from tqdm.notebook import tqdm

In [3]:
# 수집
def search(name_list):
    driver = webdriver.Chrome('chromedriver')
    driver.implicitly_wait(3)
    driver.get('https://search.naver.com/search.naver?where=news')
    
    driver.find_element_by_name('query').send_keys(name_list)
    driver.find_element_by_class_name('bt_search').click()
    
    title = []
    news_date = []
    word_data = []
    
    date_re = re.compile(r'\d\d\d\d.\d\d.\d\d.')
    
    while True:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        title_list = soup.find_all(class_='_sp_each_title')
        date = soup.find_all(class_='txt_inline')
        word = soup.find_all(class_='type01')[0].find_all('dl')

        for i in title_list:
            title.append(i.get_text())

        for dates in date:
            try:
                match = date_re.search(dates.get_text())
                news_date.append(match.group())
            except:
                date_re2 = re.compile(r'\d')
                match = date_re2.search(dates.get_text())
                m = int(match.group())
                year = str(datetime.datetime.today().year)
                month = str(datetime.datetime.today().month)
                day = str(datetime.datetime.today().day-m)
                result = year + '.' + month + '.' + day + '.'
                news_date.append(result)

        for words in word:
            word_data.append(words.find_all('dd')[1].get_text())
            
        try:
            if soup.find_all(class_='next')[0].get_text() != '다음페이지':
                break
            else: 
                driver.find_element_by_class_name('next').click()
                driver.implicitly_wait(1)
        except:
            if soup.find(class_='next') != '다음페이지':
                break
    
    df = pd.DataFrame(list(zip(title, news_date, word_data)), columns = ['title', 'date','contents'])
    driver.close()
    
    return df

In [4]:
# 전처리 2
def sentence(df):
    
    df['contents'] = df['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]"," ")
    
    stopwords = ['을', '로', '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다''의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

    hannanum = Hannanum()
    
    contents = []

    for sentence in tqdm(df['contents']):
        temp_x = []
        temp_x = hannanum.nouns(sentence)
        temp_x = [word for word in temp_x if not word in stopwords]
        contents.append(temp_x)
        
    return contents


In [5]:
# 단어 빈도수
def word_count(contents, name):
    wordlist = sum(contents, [])
    word_list = pd.Series(wordlist)
    result = word_list.value_counts()
    result.to_csv('data/기업 '+name+'.csv', mode='w')
    print(name + ' Save compelte') 

In [6]:
list_df = pd.read_csv('data/bankrup_list.csv')

In [7]:
# 회사명 리스트화
company_list = list_df['company'].tolist()

In [8]:
# 공백 없애기
company_list = [cmp_lst.strip() for cmp_lst in company_list]

In [9]:
result = pd.DataFrame(index=range(0,0), columns=['title', 'date','contents'])

In [10]:
# 리스트 기업 뉴스 한번에 스크롤링 해서 단어 빈도수 따로 저장
for i, cmp_lst in enumerate(company_list):
    print(cmp_lst + ' Start')
    df = search(cmp_lst)
    df_contents = sentence(df)
    word_count(df_contents, cmp_lst)
    result = pd.concat([result, df])

(유)동해건업 Start
(유)부강산업개발 Start
(유)세일디앤아이 Start


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-f31857c0b377>", line 4, in <module>
    df = search(cmp_lst)
  File "<ipython-input-3-878413d81082>", line 3, in search
    driver = webdriver.Chrome('chromedriver')
  File "C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 81, in __init__
    desired_capabilities=desired_capabilities)
  File "C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 157, in __init__
    self.start_session(capabilities, browser_profile)
  File "C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 252, in start_session
    response = self.execute(Command.NEW_SESSION, parameters)
  File "C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\seleni

TypeError: object of type 'NoneType' has no len()

In [11]:
result.to_csv('data/test.csv', mode='w', encoding='utf-8')